<a href="https://colab.research.google.com/github/mauricio-ms/motor-imagery-convolutional-recurrent-neural-network/blob/master/notebooks/cnn-1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verify if GPU is enabled

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
  raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


Mount Google Drive directory

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Unzip dataset

In [ ]:
!tar -xf "drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/preprocessed-csv-files.tar.xz"

Install needed packages

In [4]:
!pip install colorlog

In [5]:
import logging
import colorlog


def get_logger(name="__main__", debug=False):
    log_format = (
        "[%(levelname)s] "
        "%(asctime)s - "
        "%(name)s: "
        "%(funcName)s - "
        "%(message)s"
    )
    bold_seq = "\033[1m"
    colorlog_format = (
        f"{bold_seq} "
        "%(log_color)s "
        f"{log_format}"
    )
    colorlog.basicConfig(format=colorlog_format)

    logger = logging.getLogger(name)
    if debug:
        logger.setLevel(logging.DEBUG)
    else:
        logger.setLevel(logging.INFO)

    # Output full log
    fh = logging.FileHandler("app.log")
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter(log_format)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    return logger

In [15]:
import os
import tensorflow as tf
import numpy as np

LOGGER = get_logger("physionet.py")
TFRECORD_FILES_DIR = "drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/preprocessed-tfrecord-files"

In [16]:
def load_data(train_size=0.75, validation_size=None, n_subjects=None, **kwargs):
    LOGGER.info("Loading Physionet dataset ...")
    subjects = np.array(sorted(os.listdir(TFRECORD_FILES_DIR)))
    if n_subjects is not None:
        np.random.shuffle(subjects)
        subjects = subjects[:n_subjects]
    train_subjects, test_subjects = _train_test_split_subjects(subjects, train_size)
    if validation_size is not None:
        train_subjects, validation_subjects = _train_test_split_subjects(train_subjects, 1-validation_size)
        LOGGER.info(f"(Train, Validation, Test) Subjects = "
                    f"({len(train_subjects)}, {len(validation_subjects)}, {len(test_subjects)})")
        return _load_set(train_subjects, **kwargs), \
            _load_set(validation_subjects, **kwargs), \
            _load_set(test_subjects, **kwargs)

    LOGGER.info(f"(Train, Test) Subjects = ({len(train_subjects)}, {len(test_subjects)})")
    LOGGER.info(f"Train subjects: {train_subjects}")
    LOGGER.info(f"Test subjects: {test_subjects}")
    return _load_set(train_subjects, **kwargs), _load_set(test_subjects, **kwargs)


def _train_test_split_subjects(subjects, train_size):
    train_subjects_mask = np.random.rand(len(subjects)) < train_size
    return subjects[train_subjects_mask], subjects[~train_subjects_mask]


# TODO - convert_to_2d should define the data directory
def _load_set(subjects, n_readers=tf.data.experimental.AUTOTUNE, n_parse_threads=tf.data.experimental.AUTOTUNE, batch_size=100,
              convert_to_2d=False, expand_dim=False):
    path_files = [os.path.join(TFRECORD_FILES_DIR, subject, file_name)
                  for subject in subjects
                  for file_name in sorted(os.listdir(os.path.join(TFRECORD_FILES_DIR, subject)))]
    dataset = tf.data.Dataset.list_files(path_files)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TFRecordDataset(filepath, compression_type="GZIP"),
        cycle_length=n_readers, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda r: _preprocess(r, expand_dim=expand_dim),
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.cache()
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)


@tf.function
def _preprocess(serialized_eeg_records, expand_dim=False):
    n_channels = 64
    feature_description = {
      "X": tf.io.FixedLenFeature([n_channels], tf.float32),
      "y": tf.io.FixedLenFeature([], tf.int64)
    }
    parsed_eeg_records = tf.io.parse_example(serialized_eeg_records, FEATURE_DESCRIPTION)
    
    X = parsed_eeg_records["X"]
    y = parsed_eeg_records["y"]
    if expand_dim:
        X = X[..., np.newaxis]
    return X, y

In [17]:
from tensorflow import keras

with tf.device("/device:GPU:0"):
  train_set, test_set = load_data(expand_dim=True)

  model = keras.models.Sequential([
      keras.layers.Conv1D(32, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME",
                          input_shape=[64, 1]),
      keras.layers.Conv1D(64, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Conv1D(128, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1024, activation="relu",
                        kernel_initializer="he_normal"),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(5, activation="softmax")
  ])

  optimizer = keras.optimizers.Adam(lr=1e-4)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])

  [INFO] 2020-08-17 03:04:51,392 - physionet.py: load_data - Loading Physionet dataset ...
  [INFO] 2020-08-17 03:04:51,396 - physionet.py: load_data - (Train, Test) Subjects = (79, 29)
  [INFO] 2020-08-17 03:04:51,398 - physionet.py: load_data - Train subjects: ['S002' 'S004' 'S005' 'S006' 'S007' 'S009' 'S010' 'S013' 'S014' 'S015'
 'S016' 'S018' 'S020' 'S021' 'S024' 'S025' 'S026' 'S027' 'S029' 'S030'
 'S031' 'S032' 'S033' 'S034' 'S036' 'S037' 'S038' 'S039' 'S041' 'S042'
 'S044' 'S045' 'S046' 'S047' 'S048' 'S050' 'S051' 'S052' 'S055' 'S056'
 'S057' 'S059' 'S060' 'S061' 'S063' 'S064' 'S065' 'S066' 'S068' 'S069'
 'S072' 'S073' 'S074' 'S075' 'S076' 'S077' 'S078' 'S080' 'S081' 'S086'
 'S087' 'S088' 'S090' 'S091' 'S093' 'S094' 'S095' 'S096' 'S097' 'S098'
 'S099' 'S100' 'S101' 'S103' 'S104' 'S105' 'S106' 'S108' 'S109']
  [INFO] 2020-08-17 03:04:51,399 - physionet.py: load_data - Test subjects: ['S001' 'S003' 'S008' 'S011' 'S012' 'S017' 'S019' 'S022' 'S023' 'S028'
 'S035' 'S040' 'S043' 'S049'

In [18]:
LOGGER.info("Starting training CNN 1D model ...")
model.fit(train_set, epochs=10)
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-17 03:05:03,350 - physionet.py: <module> - Starting training CNN 1D model ...


Epoch 1/10
   4857/Unknown - 55s 11ms/step - loss: 8.2783 - accuracy: 0.5135

KeyboardInterrupt: ignored